REQUIREMENTS + UTILS

In [1]:
from scripts.read_data import read_data_sequences

LOAD DATA

In [2]:
sequences_train, sequences_test, proteins_test, y_train = read_data_sequences()

In [3]:
len(sequences_test)

1223